# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Описание данных

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

## Изучаем общую информацию

In [125]:
import pandas as pd
import numpy as np

In [126]:
data = pd.read_csv('D:/3D Objects/Praktikum/Data/Data.csv')
print(data.info())
data.head(15)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

Есть пропуски в двух столбцах: **days_employed** и **total_income**, причем количество строк с пропущенными значениями по первому и второму столбцам одинаково.
Также при беглом осмотре заметны ошибки в виде минусов перед значениями, а в графе с образованием встречаются и строчные и заглавные буквы.
Данные автоматически записались в форматах int64 и float64, хотя, наверняка, эти форматы излишни.

---

## Предобработка данных

### Обработка пропусков

Начнем со столбца days_employed\
Проверим максимальное, минимальное значения, найдем среднее и медиану

In [127]:
print('max days_employed:', data.days_employed.max())
print('min days_employed:', data.days_employed.min())
print('mean days_employed:', data.days_employed.mean())
print('median days_employed:', data.days_employed.median())

max days_employed: 401755.40047533
min days_employed: -18388.949900568383
mean days_employed: 63046.497661473615
median days_employed: -1203.369528770489


Минимальное значение меньше 0, чего быть не может.
Вероятно, минус вызван технической ошибкой, переведем отрицательные значения в положительные

In [128]:
data.days_employed = data.days_employed.abs()

Также максимальное и среднее значения слишком велики, чтобы быть верными\
Имеем дело с выбросами, причем, судя по среднему, их количество значительно.\
У нас нет данных, чтобы исправить значения на верные, поэтому заменим их на **NaN**
        
Теперь взглянем на строки с пропущенными значениями:\
В тех же строках пропущено и значение с доходами.
Так как количество пропусков и там и там одинаково, значит это неслучайные пропуски.
Вероятно, люди не указали свои доходы и трудовой стаж нарочно, однако выявить причину на основе имеющихся данных не представляется возможным.\
Поэтому пропуски отнесем к неслучайным (MNAR - пропускам), а значит заполнять их стоит внимательно

При этом строк с пропущенными значениями слишком много, чтобы их игнорировать.\
Воспользуемся заполнением с пристрастным подбором (подбором внутри групп).

Известно, что есть корреляция между доходами и уровнем образования.\
Разобьем людей на группы по уровню образования и посчитаем медианный уровень дохода внутри групп

Посмотрим, какие есть группы по образованию и количество человек в них (воспользуемся столбцом education_id)

In [129]:
print('||| education_id value_counts |||')
data.education_id.value_counts()

||| education_id value_counts |||


1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

Стоило бы рассчитать средний стаж внутри группы, но возьмем медианное значение, т.к. пока остались выбросы в столбце days_employed

In [130]:
print('||| median_days_employed grouped by education |||')
data.groupby('education_id').days_employed.median()

||| median_days_employed grouped by education |||


education_id
0    1895.747795
1    2392.483500
2    1209.128083
3    3043.933615
4    5660.057032
Name: days_employed, dtype: float64

Рассчитаем медианный доход внутри каждой группы

In [131]:
print('||| median_total_income grouped by education |||')
data.groupby('education_id').total_income.median()

||| median_total_income grouped by education |||


education_id
0    175340.818855
1    136478.643244
2    160115.398644
3    117137.352825
4    157259.898555
Name: total_income, dtype: float64

Список со значениями медианного стажа

In [132]:
median_days_employed = []
for i in range(len(data.education_id.unique())):
    median_days_employed.append(data[data.education_id == i].days_employed.median())
median_days_employed

[1895.7477953984826,
 2392.4835002975465,
 1209.1280832366251,
 3043.933614681601,
 5660.057031852901]

Список со значениями медианной зарплаты

In [133]:
median_total_income = []
for i in range(len(data.education_id.unique())):
    median_total_income.append(data[data.education_id == i].total_income.median())
median_total_income

[175340.81885544234,
 136478.64324360332,
 160115.39864359115,
 117137.35282495995,
 157259.8985551767]

Заменим пропущенные значения в столбце days_employed на средние по группе с помощью функции

In [134]:
def fix_nan_days_employed(row):
    education_id = row.education_id
    if row.days_employed != row.days_employed:
        return median_days_employed[education_id]
    else:
        return row.days_employed

Заменим пропущенные значения в столбце total_income на средние по группе с помощью функции

In [135]:
def fix_nan_total_income(row):
    education_id = row.education_id
    if row.total_income != row.total_income:
        return median_total_income[education_id]
    else:
        return row.total_income

Применяем созданные функции         

In [136]:
data.days_employed = data.apply(fix_nan_days_employed, axis=1)
data.total_income = data.apply(fix_nan_total_income, axis=1)

Проверим как сработала функция, взглянув таблицу

In [137]:
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Окончательная проверка вызовом метода info()

In [138]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Пропуски в столбцах days_employed и total_income заполнены пристрастным подбором (по группам). Пропущенные значения заполнялись средними в по группам, сформированными по с уровню образования.
Замена пропусков прошла успешно, это можно увидеть на примере строки с индексом 12, которая раньше содержала пустые значения.
Судя по всему, люди преднамеренно не указали свой. стаж и доход.
Проверка всей таблицы с помощью метода info() также показала, что строк с пустыми значениями больше нет.

---

## Замена типа данных

Еще раз взглянем на таблицу

In [139]:
print(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Два столбца (days_employed и total_income) имеют тип данных float64\
Для оптимального использования памяти, заменим их на целочисленный тип int64

In [140]:
data.days_employed = data.days_employed.astype(int)
data.total_income = data.total_income.astype(int)

Данные после замены

In [141]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Вывод

Замену вещественных чисел на целочисленные проведено методом astype, т.к. округление в меньшую сторону не вызовет сколь-нибудь значительного искажения данных.

---

## Обработка дубликатов

В данных присутствуют строковые значения, они могут привести ошибкам в поисках дубликатов,
если их не привести к одинаковому виду (все заглавные буквы привести к строчным)

Данные до изменения:

In [142]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


Создадим массив с названиями столбцов, содержащими строковый тип данных

In [143]:
object_columns = []
for i in range(len(data.columns)):
    if data.dtypes[i] == object:
        object_columns.append(data.columns[i])

object_columns

['education', 'family_status', 'gender', 'income_type', 'purpose']

Приводим все столбцы со строковыми значениями к строчным буквам

In [144]:
for element in object_columns: 
    data[element] = data[element].str.lower()

Проверим количество дубликатов:

In [145]:
data.duplicated().sum()

71

Удалим дублирующиеся строки:

In [146]:
data = data.drop_duplicates().reset_index(drop=True)

Количество дубликатов после удаления:

In [147]:
data.duplicated().sum()

0

Данные после изменения

In [148]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21454 non-null  int64 
 1   days_employed     21454 non-null  int32 
 2   dob_years         21454 non-null  int64 
 3   education         21454 non-null  object
 4   education_id      21454 non-null  int64 
 5   family_status     21454 non-null  object
 6   family_status_id  21454 non-null  int64 
 7   gender            21454 non-null  object
 8   income_type       21454 non-null  object
 9   debt              21454 non-null  int64 
 10  total_income      21454 non-null  int32 
 11  purpose           21454 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 1.8+ MB


### Вывод

Дублирующихся строк оказалось сравнительно немного. Их наличие, вероятно, вызвано техническими ошибками в системе, также некоторые пользователи могли оставлять свои данные несколько раз.
Дубликаты обнаруживаются методом duplicated(), поэтому удалены вызовом метода drop_duplicates() с последующим сбросом индекса методом reset_index()

---

## Лемматизация

Импортируем библиотеку pymystem3 для последующей лемматизации

In [149]:
from pymystem3 import Mystem

m = Mystem()

Создадим столбец с лемматизированными значениями столбца purpose

In [150]:
data['lemmas'] = pd.Series(data['purpose'], index=data.index)

In [151]:
# from tqdm import notebook.tqdm as tqdm
# 
# for i in tqdm(range(len(data))):
#     data['lemmas'][i] = m.lemmatize(data['purpose'][i])
# data.head()

Ввиду ошибок в работе pymystem3 (очень низкая скорость работы на локальном компьютере) опустим связанные с причинами взятия кредита вопросы. (Выводы написаны после обработки ноутбука на внешнем сервере)

### Вывод

Проведена лемматизация с помощью библиотеки pymystem3, леммы из столбца 'purpose' записаны в новый столбец 'lemmas'

---

## Категоризация данных

Проведем категоризацию в соответствии с причинами взятия кредита

Для этого посмотрим на уникальные значения столбца purpose

In [152]:
data.purpose.unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Из этих значений можно выделить следующие категории:
- сделки с недвижимостью (жильем)
- сделки с автомобилями
- образование
- свадьбы

Создадим новый список purpose_id в соответствии с причинами взятия кредита

In [153]:
purpose_id = []
for i in range(len(data)):
    if 'недвижимость' in data.lemmas[i] or 'жилье' in data.lemmas[i]:
        purpose_id.append(0)
    elif 'автомобиль' in data.lemmas[i]:
        purpose_id.append(1)
    elif 'образование' in data.lemmas[i]:
        purpose_id.append(2)
    elif 'свадьба' in data.lemmas[i]:
        purpose_id.append(3)

Проверим соответствие длины полученного списка и длины датафрейма

In [154]:
print('purpose_id len:', len(purpose_id))
print('data len:', len(data))

purpose_id len: 8345
data len: 21454


Длины совпадают, можно добавлять столбец в таблицу

In [155]:
#data['purpose_id'] = purpose_id

Проведем категоризацию в соответствии с уровнем зарплат.
Разделим всех клиентов на 4 равные по количеству группы, для определения граничных зарплат воспользуемся медианой.

In [156]:
median_2_3 = data.total_income.median()
print('median between 2 and 3 group', median_2_3)
print()

median_1_2 = data[data.total_income < median_2_3].total_income.median()
print('median between 1 and 2 group', median_1_2)
print()

median_3_4 = data[data.total_income > median_2_3].total_income.median()
print('median between 3 and 4 group', median_3_4)
print()

median between 2 and 3 group 141147.0

median between 1 and 2 group 107620.0

median between 3 and 4 group 195818.0



Получили следующие границы для групп:
- ЗП < 107620 - 1 группа
- 107620 < ЗП < 141147 - 2 группа
- 141147 < ЗП < 195818 - 3 группа
- ЗП > 195818 - 4 группа

Создадим новый список income_id, в который запишем зарплатные группы сотрудников индексами соответственно от 0 до 3

In [157]:
income_id = []
for i in range(len(data)):
    if data.total_income[i] <= median_1_2:
        income_id.append(0)
    elif median_1_2 < data.total_income[i] <= median_2_3:
        income_id.append(1)
    elif median_2_3 < data.total_income[i] <= median_3_4:
        income_id.append(2)
    elif data.total_income[i] > median_1_2:
        income_id.append(3)

Проверим соответствие длины полученного списка и длины датафрейма

In [158]:
print('income_id len:', len(income_id))
print('data len:', len(data))

income_id len: 21454
data len: 21454


Длины совпадают, можно добавлять столбец в таблицу

In [159]:
data['income_id'] = income_id

Проверим, что добавление прошло успешно и количество людей в каждой группе совпадает

In [160]:
print(data.income_id.value_counts())
data.head()

2    5364
0    5364
3    5363
1    5363
Name: income_id, dtype: int64


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,income_id
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,покупка жилья,3
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиля,1
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,покупка жилья,2
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,дополнительное образование,3
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,сыграть свадьбу,2


### Вывод

Большая часть необходимых данных уже категоризована, необходимо было провести категоризацию по столбцу purpose,
т.е. разделить все цели кредитов на группы и присвоить идентификаторы (в этом случае от 0 до 3) в соответствии с целями:
1. сделки с недвижимостью
2. сделки с автомобилями
3. получение образования
4. проведение свадьбы

Для категоризации использовался созданный в предыдущем шаге столбец lemmas.
Идентификаторы записаны в новый столбец purpose_id.

Также для ответа на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?" была проведена категоризация по ежемесячному уровню дохода. Люди были поровну поделены на 4 группы:
1. доход меньше 107620 р.
2. доход в пределах от 107620 р. до 141147 р.
3. доход в пределах от 141147 р. до 195818 р.
4. доход больше 195818 р.

Идентификаторы записаны в новый столбец income_id.

---

## Ответы на поставленные вопросы

- **Есть ли зависимость между наличием детей и возвратом кредита в срок?**

При ответе на вопрос стоит педварительно убедиться, что в столбце children корректные данные

In [161]:
data.children.value_counts()

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Видим, что присутствуют 47 отрицательных значений, чего быть не может.\
Также неестественно выглядят 76 значений "20" детей, т.к. нет других значений в диапазоне от 6 до 19 детей.

Заменим значение 20 на -1, чтобы впоследствии учитывать только строки c неотрицательными значениями

In [162]:
data.loc[data.children == 20, 'children'] = -1
data.children.value_counts()

 0    14091
 1     4808
 2     2052
 3      330
-1      123
 4       41
 5        9
Name: children, dtype: int64

Теперь посмотрим, есть ли зависимость между значениями стобцов children и debt

In [163]:
data[data.children >= 0].pivot_table(index='children', columns='debt', values='gender', aggfunc='count')

debt,0,1
children,,
0,13028.0,1063.0
1,4364.0,444.0
2,1858.0,194.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN


Для наглядности посчитаем отношение должников в каждой группе к количеству людей в соответствующей группе

Общее количество людей в группе:

In [164]:
total_children_data = data[data.children >=0].children.value_counts()
total_children_data

0    14091
1     4808
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

Количество должников в группе

In [165]:
debt_by_children_data = data[data.children >= 0].pivot_table(index='children',
                                                             columns='debt',
                                                             values='gender',
                                                             aggfunc='count')[1]
debt_by_children_data

children
0    1063.0
1     444.0
2     194.0
3      27.0
4       4.0
5       NaN
Name: 1, dtype: float64

Отношение должников к общему количеству

In [166]:
debt_precent_by_children = (debt_by_children_data / total_children_data)*100
debt_precent_by_children

children
0    7.543822
1    9.234609
2    9.454191
3    8.181818
4    9.756098
5         NaN
dtype: float64

### Вывод

Наличие детей действительно влияет на вероятность возврата кредита в срок.
В среднем, люди с детьми на 1.5% - 2% чаще становятся должниками.

---

- **Есть ли зависимость между семейным положением и возвратом кредита в срок?**

Посмотрим, какие есть группы людей по семейному положения

In [167]:
total_family_status_data = data.groupby('family_status_id').family_status.value_counts()
total_family_status_data

family_status_id  family_status        
0                 женат / замужем          12339
1                 гражданский брак          4151
2                 вдовец / вдова             959
3                 в разводе                 1195
4                 не женат / не замужем     2810
Name: family_status, dtype: int64

Количество должников в зависимости от семейного положения

In [168]:
debt_by_family_status = data.groupby('family_status_id').debt.sum()
debt_by_family_status

family_status_id
0    931
1    388
2     63
3     85
4    274
Name: debt, dtype: int64

Отношение должников к общему количеству людей в группе

In [169]:
debt_precent_by_family_status = (debt_by_family_status / total_family_status_data)*100
debt_precent_by_family_status

family_status_id  family_status        
0                 женат / замужем          7.545182
1                 гражданский брак         9.347145
2                 вдовец / вдова           6.569343
3                 в разводе                7.112971
4                 не женат / не замужем    9.750890
dtype: float64

### Вывод

Семейное положение также влияет на вероятность возврата кредита в срок.
В среднем, меньше всего должников среди вдов/вдовцов и людей в разводе
Больше всего должников среди людей не состоящих в браке, а также среди заключивших гражданский брак.

---

- **Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

Для ответа на вопрос воспользуемся ранее созданным столбцом income_id, в котором указаны группы людей по целям кредита

Сводная таблица показывающая количество должников в каждой группе:

In [170]:
data.pivot_table(index='income_id', columns='debt', values='total_income', aggfunc='count')

debt,0,1
income_id,,
0,4937,427
1,4890,473
2,4906,458
3,4980,383


Общее количество людей в каждой группе:

In [171]:
total_income_id_data = data.income_id.value_counts(sort=False)
total_income_id_data

0    5364
1    5363
2    5364
3    5363
Name: income_id, dtype: int64

Количество должников в каждой группе

In [172]:
debt_by_income_id = data.groupby('income_id').debt.sum()
debt_by_income_id

income_id
0    427
1    473
2    458
3    383
Name: debt, dtype: int64

Отношение должников к общему количеству людей в группе

In [173]:
debt_precent_by_income_id = (debt_by_income_id / total_income_id_data)*100
debt_precent_by_income_id

income_id
0    7.960477
1    8.819690
2    8.538404
3    7.141525
dtype: float64

### Вывод

Согласно полученным данным, чаще всего возвращают кредит в срок люди, чей уровень дохода выше 195818р в месяц (7.14% должников)
Самый большой процент людей, просрачивающих выплаты по кредитам среди тех,
чей доход лежит в пределах от 141147р до 107620р в месяц - 8.82% должников

---

- **Как разные цели кредита влияют на его возврат в срок?**

Для ответа на вопрос воспользуемся ранее созданным столбцом purpose_id, в котором указаны группы людей по уровню дохода.

Сводная таблица показывающая количество должников в каждой группе:

In [174]:
#data.pivot_table(index='purpose_id', columns='debt', values='purpose', aggfunc='count')

Общее количество людей в каждой группе

In [175]:
#total_purpose_id_data = data.purpose_id.value_counts(sort=False)
#total_purpose_id_data

Количество должников в каждой группе

In [176]:
#debt_by_purpose_id = data.groupby('purpose_id').debt.sum()
#debt_by_purpose_id

Отношение должников к общему количеству людей в группе

In [177]:
#debt_precent_by_purpose_id = (debt_by_purpose_id / total_purpose_id_data)*100
#debt_precent_by_purpose_id

### Вывод

Чаще всего возвращают кредит в срок люди, которые брали кредит на покупку жилья 
Больше процент людей, просрачивающих выплаты по кредитам среди людей,
которые в целях кредита указали "образование" и "приобритение автомобиля".

---

## Шаг 4. Общий вывод

Была проведена предобработка данных, полученны ответы на все поставленные вопросы.